### Importing the Libraries

In [37]:
import pandas as pd
#import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules
from scipy.sparse import csr_matrix

### Preprocessing

In [38]:
# Fetching our data
tData = pd.read_csv("TransactionOrder2.csv")

In [39]:
tData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7421 entries, 0 to 7420
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   trans_ID                7421 non-null   object 
 1   tr_date                 7421 non-null   object 
 2   item_ID                 7421 non-null   object 
 3   item_description        7421 non-null   object 
 4   item_cat_1_ID           7395 non-null   object 
 5   base_sku                7421 non-null   object 
 6   total_amt               7421 non-null   float64
 7   Sheet1.billto_ID        7421 non-null   object 
 8   Sheet1.billto_name      6827 non-null   object 
 9   Sheet1.billto_tin       6216 non-null   float64
 10  Sheet1 (3).description  7395 non-null   object 
dtypes: float64(2), object(9)
memory usage: 637.9+ KB


In [40]:
# Only taking the necessary columns
df = tData[["trans_ID", "item_description"]]

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7421 entries, 0 to 7420
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   trans_ID          7421 non-null   object
 1   item_description  7421 non-null   object
dtypes: object(2)
memory usage: 116.1+ KB


In [42]:
# converting the data to an item basket
# Grouping items by transaction ID, keeping only item descriptions
item_baskets = df.groupby("trans_ID")["item_description"].apply(list)

# Print the item baskets
print(item_baskets)

trans_ID
001-04-0000000001                       [RS extra virgin olive oil 1l**12, Local Vinge...
001-04-0000000002                       [Fanta pineapple  500ml**, Elsa kolo, Fahm wat...
001-04-0000000003                       [coffee 2, Elsa kolo, Basket 25lit, zera roach...
001-04-0000000004                       [Tissue paper**48, euro facial tissue, crown d...
001-04-0000000005                       [Mara Chick peas 400gm, Mara processed peas 40...
                                                              ...                        
fe6d8b8b-34f6-41b7-b75e-362467627a7b                                              [broom]
fef1c384-bd0c-463d-8593-76c2047f025b    [Cheese Berta, BUTTER((*5, special top side me...
ff41967b-b9f7-4dd9-a760-7f5deeb7941e                                              [candy]
ff42c2be-ff7c-4fbe-aa2b-39441cbea1f5    [vitrac apple juice 1li, vitrac pineapple  jui...
ffc18917-0022-4387-a4b7-9b7bc5a90ff1                                           [ani pant]
N

### Transforming the item baskets to a boolean matrix

In [43]:
unique_items = pd.Series(df['item_description'].unique())

In [44]:
boolean_matrix = pd.DataFrame(False, index=df['trans_ID'].unique(), columns=unique_items)

In [45]:
for trans_ID, items in item_baskets.items():
    for item in items:
        boolean_matrix.loc[trans_ID, item] = True

In [46]:
boolean_matrix_sparse = csr_matrix(boolean_matrix)

In [47]:
print(boolean_matrix.head())

                                      laundry plastic  \
1c3dec63-5d6a-4aaa-a47d-f515180cac91             True   
273e062f-29dc-48c4-b389-213433718e04            False   
2a773103-c741-4caf-8702-9822251ad8d2            False   
6d392ce1-6864-4642-9235-1b39fb96830c            False   
125d5ab4-4cd1-4913-9063-24b653c00a6a            False   

                                      Romoli balsamico vingar 500ml  \
1c3dec63-5d6a-4aaa-a47d-f515180cac91                          False   
273e062f-29dc-48c4-b389-213433718e04                           True   
2a773103-c741-4caf-8702-9822251ad8d2                          False   
6d392ce1-6864-4642-9235-1b39fb96830c                          False   
125d5ab4-4cd1-4913-9063-24b653c00a6a                          False   

                                      bakerd brown sugar sticks  \
1c3dec63-5d6a-4aaa-a47d-f515180cac91                      False   
273e062f-29dc-48c4-b389-213433718e04                       True   
2a773103-c741-4caf-8702-98222

### Applying Apriori

In [48]:
frequent_itemsets = apriori(boolean_matrix, min_support=0.01, use_colnames=True)

In [34]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
print(rules)

                                          antecedents  \
0                                        (Flour 50kg)   
1                           (quaker white oats 500gm)   
2                            (rosa peanut butter 1kg)   
3                           (nido powder milk 2500gm)   
4                                        (BUTTER((*5)   
..                                                ...   
90             (farm chicken, farm egg, Cheese Berta)   
91  (farm chicken, special top side meat, Cheese B...   
92    (farm egg, special top side meat, Cheese Berta)   
93                           (farm chicken, farm egg)   
94                       (farm chicken, Cheese Berta)   

                              consequents  antecedent support  \
0                            (BUTTER((*5)            0.028777   
1                      (knorr-all-in-one)            0.031175   
2               (quaker white oats 500gm)            0.017586   
3                            (BUTTER((*5)            0.

In [49]:
rules_df = pd.DataFrame(rules)

rules_df = rules_df.drop(['lift','leverage','conviction','zhangs_metric'], axis = 1)

In [36]:
rules_df.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence
0,(Flour 50kg),(BUTTER((*5),0.028777,0.056755,0.016787,0.583333
1,(quaker white oats 500gm),(knorr-all-in-one),0.031175,0.032774,0.015987,0.512821
2,(rosa peanut butter 1kg),(quaker white oats 500gm),0.017586,0.031175,0.011990,0.681818
3,(nido powder milk 2500gm),(BUTTER((*5),0.015987,0.056755,0.010392,0.650000
4,(BUTTER((*5),(special top side meat),0.056755,0.108713,0.035971,0.633803
5,(farm chicken),(special top side meat),0.036771,0.108713,0.025580,0.695652
6,(niylperch fish),(special top side meat),0.026379,0.108713,0.018385,0.696970
7,(Cheese Berta),(special top side meat),0.055156,0.108713,0.036771,0.666667
8,(farm egg),(special top side meat),0.055156,0.108713,0.028777,0.521739
9,(star lime vim 500ml *12),(smart bleach 800ml),0.015188,0.027978,0.010392,0.684211


In [16]:
pd.set_option('display.max_rows', None)